In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error

In [7]:
# Função para preparar a visão de diárias de UTI por mês/ano de um estabelecimento específico
def prepare_time_series_for_estab(data, estab_name):
    # Filtrar o dataframe pelo estabelecimento desejado
    data_estab = data[data['Hospital'].str.contains(estab_name, na=False, case=False)]
    
    # Se não encontrar o estabelecimento, retorne None
    if data_estab.empty:
        print(f"Estabelecimento '{estab_name}' não encontrado.")
        return None
    
    # Removendo colunas desnecessárias e transformando o índice
    data_estab = data_estab.drop(['Estab.Saúde-Cidade', 'Criterio'], axis=1)
    data_estab.columns = pd.to_datetime(data_estab.columns, errors='coerce')
    data_estab = data_estab.transpose()
    
    # Limpar os dados, substituir '-' por NaN e converter para float
    data_estab.replace('-', np.nan, inplace=True)
    data_estab = data_estab.apply(pd.to_numeric, errors='coerce')
    
    # Preencher valores ausentes com a média da coluna
    data_estab.fillna(data_estab.mean(), inplace=True)
    
    # Sumarizar os dados para obter o total por mês
    data_estab_sum = data_estab.sum(axis=1)
    
    # Definir uma frequência mensal para o índice
    data_estab_sum.index = pd.date_range(start=data_estab_sum.index[0], periods=len(data_estab_sum), freq='M')
    
    return data_estab_sum

# Função de previsão usando SARIMAX com auto_arima
def forecast_time_series(data, steps=12):
    # Use auto_arima para encontrar os melhores hiperparâmetros do modelo SARIMA
    model = auto_arima(data, seasonal=True, m=12, stepwise=True, trace=True, error_action='ignore',
                      suppress_warnings=True, max_order=None)
    
    # Ajustar o modelo SARIMAX com os hiperparâmetros encontrados
    sarimax_model = SARIMAX(data, order=model.order, seasonal_order=model.seasonal_order)
    fitted_model = sarimax_model.fit(disp=False)
    
    # Fazer previsões
    forecast = fitted_model.get_forecast(steps=steps)
    forecast_index = pd.date_range(data.index[-1] + DateOffset(months=1), periods=steps, freq='M')
    forecast_values = forecast.predicted_mean
    forecast_conf_int = forecast.conf_int()
    
    # Construir dataframe de previsão
    forecast_df = pd.DataFrame({
        'Forecast': forecast_values,
        'Lower CI': forecast_conf_int.iloc[:, 0],
        'Upper CI': forecast_conf_int.iloc[:, 1]
    }, index=forecast_index)
    
    return forecast_df

# Função de previsão usando Prophet
def forecast_time_series_prophet(data):
    # Preparar o dataframe para o Prophet
    data_prophet = pd.DataFrame({
        'ds': data.index,
        'y': data.values
    })
    
    # Ajustar o modelo Prophet
    prophet_model = Prophet()
    prophet_model.fit(data_prophet)
    
    # Fazer previsões
    future = prophet_model.make_future_dataframe(periods=12, freq='M')
    forecast = prophet_model.predict(future)
    
    return forecast.set_index('ds')['yhat']

# Função para calcular o MSE
def calculate_mse(actual, predicted):
    return mean_squared_error(actual, predicted)

# Carregar o arquivo Excel
file_path = r'c:\Users\Thiago\Downloads\Diarias de UTI por Mesano de competencia segundo Estab.Saude-Cidade .xlsx'
data = pd.read_excel(file_path)

# Obter os nomes únicos dos estabelecimentos de saúde
estab_names = data['Estab.Saúde-Cidade'].unique()


# Iterar sobre os nomes dos estabelecimentos e fazer previsões para cada um
for estab_name in estab_names:
    print(f"Previsão para o estabelecimento '{estab_name}':")
    # Preparar a série temporal para o estabelecimento específico
    estab_time_series = prepare_time_series_for_estab(data, estab_name)

    # Se estab_time_series não for None, faça a previsão
    if estab_time_series is not None:
        # Previsão com SARIMAX
        estab_forecast_sarimax = forecast_time_series(estab_time_series)
        # Previsão com Prophet
        estab_forecast_prophet = forecast_time_series_prophet(estab_time_series)
        
        # Calcular o MSE para cada previsão
        mse_sarimax = calculate_mse(estab_time_series[-12:], estab_forecast_sarimax['Forecast'])
        mse_prophet = calculate_mse(estab_time_series[-12:], estab_forecast_prophet[-12:])
        
        # Imprimir os MSEs
        print(f"MSE do modelo SARIMAX: {mse_sarimax}")
        print(f"MSE do modelo Prophet: {mse_prophet}")
        
        # Comparar os MSEs e escolher o melhor modelo
        if mse_sarimax < mse_prophet:
            print("O modelo SARIMAX é o melhor para este estabelecimento.")
        else:
            print("O modelo Prophet é o melhor para este estabelecimento.")
        
        # Plotar as previsões
        plt.figure(figsize=(10, 6))
        plt.plot(estab_time_series, label='Dados originais')
        plt.plot(estab_forecast_sarimax['Forecast'], label=f'Previsão SARIMAX (MSE: {mse_sarimax:.2f})')
        plt.plot(estab_forecast_prophet[-12:], label=f'Previsão Prophet (MSE: {mse_prophet:.2f})')
        plt.legend()
        plt.title(f"Previsões para o estabelecimento '{estab_name}'")
        plt.show()

c:\Users\DuduCuco\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\DuduCuco\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\DuduCuco\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\DuduCuco\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters e

LinAlgError: Schur decomposition solver error.